In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
# Set up credentials:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4o-mini"
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
def chat_completion_request(client, messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

def create_message(role, content):
    return {'role': role, 'content': content}

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_win_rate",
            "description": "Get the win rate over a time period. We define win rate as won deals divided by total deals.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The company that the user works for.",
                    },
                    "start_dt": {
                        "type": "string",
                        "description": "The start date of the date range for calculating win rate.",
                    },
                    "end_dt": {
                        "type": "string",
                        "description": "The end date of the date range for calculating win rate.",
                    },
                },
                "required": ["company_name", "start_dt", "end_dt"],
            },
        }
    },
]

In [5]:


def get_win_rate(company_name, start_dt, end_dt):
    return f"Get the win rate for the company - {company_name}, from {start_dt} to {end_dt}!"

def execute_function_call(message):
    args = json.loads(message.tool_calls[0].function.arguments)
    if message.tool_calls[0].function.name == "get_win_rate":
        results = get_win_rate(args["company_name"], args["start_dt"], args["end_dt"] )
    else:
        results = f"Error: function {message.tool_calls[0].function.name} does not exist"
    return results

In [6]:
# Define messages

messages = []
messages.append(
    create_message(
        "system",
        "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    )
)
messages.append(create_message("user", "What is my team's win rate?"))

In [7]:
# Submit response

chat_response = chat_completion_request(
    client, messages, tools=tools
)
messages.append(chat_response.choices[0].message)
print(chat_response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Could you please provide me with your company's name and the date range for which you would like to calculate the win rate?", refusal=None, role='assistant', function_call=None, tool_calls=None))


In [8]:
# User replies with additional details:
messages.append(
    create_message(
        "user",
        "Sure. I work for Twilio and let's calulate from the start of 2024 until the end of Q3."
    )
)

In [9]:
# The Model responds:
chat_response = chat_completion_request(
    client, messages, tools=tools
)
messages.append(chat_response.choices[0].message)
print(chat_response.choices[0])

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4zFuV5jIqksC8oKIC177M8df', function=Function(arguments='{"company_name": "Twilio", "start_dt": "2024-01-01", "end_dt": "2024-09-30"}', name='get_win_rate'), type='function')]))


In [10]:
latest_response = chat_response.choices[0]

if latest_response.finish_reason == "tool_calls":
    latest_msg = latest_response.message
    results = execute_function_call(latest_msg)
    messages.append({"role": "function",
                     "tool_call_id": latest_msg.tool_calls[0].id,
                     "name": latest_msg.tool_calls[0].function.name,
                     "content": results
                     })

In [11]:
messages

[{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user', 'content': "What is my team's win rate?"},
 ChatCompletionMessage(content="Could you please provide me with your company's name and the date range for which you would like to calculate the win rate?", refusal=None, role='assistant', function_call=None, tool_calls=None),
 {'role': 'user',
  'content': "Sure. I work for Twilio and let's calulate from the start of 2024 until the end of Q3."},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4zFuV5jIqksC8oKIC177M8df', function=Function(arguments='{"company_name": "Twilio", "start_dt": "2024-01-01", "end_dt": "2024-09-30"}', name='get_win_rate'), type='function')]),
 {'role': 'function',
  'tool_call_id': 'call_4zFuV5jIqksC8oKIC177M8df',
  'name': 'get_win_rate',
  'content': 'Get